In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Major Project/train.csv')

In [ ]:
nan_rows = df[df.isnull().any(1)]
print (nan_rows)

            id  ...  is_duplicate
105780  105780  ...             0
201841  201841  ...             0
363362  363362  ...             0

[3 rows x 6 columns]


In [ ]:
df = df.dropna(axis=0)
nan_rows = df[df.isnull().any(1)]
print (nan_rows)

Empty DataFrame
Columns: [id, qid1, qid2, question1, question2, is_duplicate]
Index: []


In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
X_train = df.iloc[:,:5].values
Y_train = df.iloc[:,5:].values

In [ ]:
s1 = X_train[:,3]
s2 = X_train[:,4]

In [ ]:
import nltk
nltk.download('punkt')
def tokenize(s):
    tokens = []
    tokens = [word_tokenize(str(sentence)) for sentence in s]

    rm1 = []
    for w in tokens:
        sm = re.sub('[^A-Za-z]',' ', str(w))
        x = re.split("\s", sm)
        rm1.append(x)
        
    return rm1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def lower_case(s):
    #Removing whitespaces    
    for sent in s:
        while '' in sent:
            sent.remove('')

    # Lowercasing
    low = []
    for i in s:
        i = [x.lower() for x in i]
        low.append(i)
        
    return low

In [ ]:
def lemmatize(s):
    lemma = []
    wnl = WordNetLemmatizer()
    for doc in s:
        tokens = [wnl.lemmatize(w) for w in doc]
        lemma.append(tokens)

    # Removing Stopwords
    filter_words = []
    Stopwords = set(stopwords.words('english'))

    #ab = spell('nd')
    for sent in lemma:
        tokens = [w for w in sent if w not in Stopwords]
        filter_words.append(tokens)

    space = ' ' 
    sentences = []
    for sentence in filter_words:
        sentences.append(space.join(sentence))
        
    return sentences

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
sw = set(stopwords.words("english"))

# remove stop words
def filter_words(word_list):
    useful_words = [ w for w in word_list if w not in sw ]
    return(useful_words)

def preprocess_data(dataset):
    data = dataset.copy()
    t = []
    for i in data:
        sentence = str(i)
        sentence = sentence.lower()
        sentence = re.sub(r'@\w+ | http | #\w+', '', sentence)
        t.append(sentence)

    #Removing punctuations, special characters and lemmatizing words to their base form
    data_train = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]',' ',text)) for text in lis]) for lis in t]
    
    a=[]
    for text in data_train:
        word_list = word_tokenize(text)
        # text=filter_words(word_list)
        # a.append(text)  
        a.append(word_list)
    
    train_text = []
    for i in a:
      train_text.append(' '.join(i))

    return train_text

In [ ]:
# question1 = (list(df['question1'].values.astype(str)))
# question2 = (list(df['question2'].values.astype(str)))

In [ ]:
# cleaned_q1 = preprocess_data(question1)
# cleaned_q2 = preprocess_data(question2)

In [ ]:
# len(cleaned_q1+cleaned_q2)

In [ ]:
# MAX_NB_WORDS = 200000
# tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
# tokenizer.fit_on_texts(cleaned_q1+cleaned_q2)

In [ ]:
MAX_NB_WORDS = 200000
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(list(df['question1'].values.astype(str))+list(df['question2'].values.astype(str)))

In [ ]:
# X_train_q1 = tokenizer.texts_to_sequences(np.array(listq1))
X_train_q1 = tokenizer.texts_to_sequences(df['question1'].values.astype(str))
X_train_q1 = pad_sequences(X_train_q1, maxlen = 30, padding='post')

# X_train_q2 = tokenizer.texts_to_sequences(np.array(listq2))
X_train_q2 = tokenizer.texts_to_sequences(df['question2'].values.astype(str))
X_train_q2 = pad_sequences(X_train_q2, maxlen = 30, padding='post')

In [ ]:
len(X_train_q1)

404287

In [ ]:
import pickle

# saving
with open('/content/drive/MyDrive/Major Project/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # loading
# with open('tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)

In [ ]:
# print(cleaned_q1[0])
# # X_train_q1 = tokenizer.texts_to_sequences(np.array(listq1))
# print("---------------------------------------------------------------------------------------------")
# X_train_q1 = tokenizer.texts_to_sequences(cleaned_q1)
# print(X_train_q1[0])
# print("---------------------------------------------------------------------------------------------")
# X_train_q1 = pad_sequences(X_train_q1, maxlen = 30, padding='post')
# print(X_train_q1[0])
# print("---------------------------------------------------------------------------------------------")

# # X_train_q2 = tokenizer.texts_to_sequences(np.array(listq2))
# X_train_q2 = tokenizer.texts_to_sequences(cleaned_q2)
# X_train_q2 = pad_sequences(X_train_q2, maxlen = 30, padding='post')

In [ ]:
# i = 0
# print(X_train_q1[i])
# print(type(X_train_q1))
# print(len(X_train_q1))
# print(type(X_train_q1[i]))
# print(list(df['question1'])[i])
# print(cleaned_q1[i])

In [ ]:
word_index = tokenizer.word_index

In [ ]:
embedding_index = {}
with open('/content/drive/MyDrive/Major Project/glove.6B.200d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [ ]:
embedding_matrix = np.random.random((len(word_index)+1, 200))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# Model for Q1
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
model_q1 = tf.keras.Sequential()
model_q1.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 200,
                      weights = [embedding_matrix],
                      input_length = 30))
model_q1.add(LSTM(128, activation = 'tanh', return_sequences = True))
model_q1.add(Dropout(0.2))
model_q1.add(LSTM(128, return_sequences = True))
model_q1.add(LSTM(128))
model_q1.add(Dense(60, activation = 'tanh'))
model_q1.add(Dense(2, activation = 'sigmoid'))

In [ ]:
# Model for Q2
model_q2 = tf.keras.Sequential()
model_q2.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 200,
                      weights = [embedding_matrix],
                      input_length = 30))
model_q2.add(LSTM(128, activation = 'tanh', return_sequences = True))
model_q2.add(Dropout(0.2))
model_q2.add(LSTM(128, return_sequences = True))
model_q2.add(LSTM(128))
model_q2.add(Dense(60, activation = 'tanh'))
model_q2.add(Dense(2, activation = 'sigmoid'))

In [ ]:
# Merging the output of the two models,i.e, model_q1 and model_q2
mergedOut = Multiply()([model_q1.output, model_q2.output])

mergedOut = Flatten()(mergedOut)
mergedOut = Dense(100, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.2)(mergedOut)
mergedOut = Dense(50, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.2)(mergedOut)
mergedOut = Dense(2, activation = 'sigmoid')(mergedOut)

In [ ]:
filepath = "/content/drive/MyDrive/Major Project/saved_weight1/model_{epoch:02d}"
new_model = Model([model_q1.input, model_q2.input], mergedOut)
new_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',
                 metrics = ['accuracy'])
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
history = new_model.fit([X_train_q1,X_train_q2],Y_train, batch_size = 2000, epochs = 20,steps_per_epoch=203, validation_split=0.2,shuffle=True,callbacks=[checkpoint],use_multiprocessing=True)

Epoch 1/20
203/203 [==============================] - 85s 352ms/step - loss: 0.6470 - accuracy: 0.6293 - val_loss: 0.5849 - val_accuracy: 0.7010


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Major Project/saved_weight1/model_01/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Major Project/saved_weight1/model_01/assets
/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
203/203 [==============================] - 69s 341ms/step - loss: 0.5544 - accuracy: 0.7230 - val_loss: 0.5297 - val_accuracy: 0.7393


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Major Project/saved_weight1/model_02/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Major Project/saved_weight1/model_15/assets
/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 16/20
195/203 [===========================>..] - ETA: 2s - loss: 0.1187 - accuracy: 0.9511WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 4060 batches). You may need to use the repeat() function when building your dataset.


203/203 [==============================] - 66s 327ms/step - loss: 0.1187 - accuracy: 0.9511 - val_loss: 1.0215 - val_accuracy: 0.7696


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Major Project/saved_weight1/model_16/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Major Project/saved_weight1/model_16/assets
/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
from keras.models import load_model
import pickle


# loading
with open('/content/drive/MyDrive/Major Project/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
models = load_model('/content/drive/MyDrive/Major Project/saved_weight1/model_10')

In [ ]:
question1 = (list(df['question1'].values.astype(str)))
question2 = (list(df['question2'].values.astype(str)))

In [ ]:
question1 = "should i own a cat"
question2 = "is cat good house pet"
testq1 = [question1]
testq2 = [question2]
test_q1 = tokenizer.texts_to_sequences(testq1)
test_q1 = pad_sequences(test_q1 , maxlen = 30, padding='post')

# X_train_q2 = tokenizer.texts_to_sequences(np.array(listq2))
test_q2 = tokenizer.texts_to_sequences(testq2)
test_q2 = pad_sequences(test_q2, maxlen = 30, padding='post')
models.predict(x=[test_q1,test_q2])

array([[0.45207882, 0.5902355 ]], dtype=float32)

In [ ]:
i = 5
question1[i]

'Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?'

In [ ]:
question2[i]

"I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?"

In [ ]:
list(df['is_duplicate'])[i]

1

In [ ]:
testq1 = [question1[i]]
testq2 = [question2[i]]
test_q1 = tokenizer.texts_to_sequences(testq1)
test_q1 = pad_sequences(test_q1 , maxlen = 30, padding='post')

# X_train_q2 = tokenizer.texts_to_sequences(np.array(listq2))
test_q2 = tokenizer.texts_to_sequences(testq2)
test_q2 = pad_sequences(test_q2, maxlen = 30, padding='post')

In [ ]:
models.predict(x=[test_q1,test_q2])

array([[0.00632011, 0.9986731 ]], dtype=float32)

In [ ]:
test_q1

array([[  16,    1,  152,  196,    3, 1984,   62,   16,   59,   19,   47,
        1984,   45,  196,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)